In [1]:
from pymongo import MongoClient
import pandas as pd
import numpy as np

In [2]:
def _connect_mongo():
    """ A util for making a connection to mongo """
    host = "localhost"
    port = 27017
    username = ""
    password = ""
    db = "5gopt"

    if username and password:
        mongo_uri = 'mongodb://%s:%s@%s:%s/%s' % (username, password, host, port, db)
        conn = MongoClient(mongo_uri)
    else:
        conn = MongoClient(host, port)


    return conn[db]

In [3]:
def collection_read_mongo(collection, query={}, no_id = True):
    db = _connect_mongo()
    cursor = db[collection].find(query)
    df = pd.DataFrame(list(cursor))

    if no_id:
        try:
            del df["_id"]
        except:
            pass
    return df

In [5]:
data = collection_read_mongo(collection="main_file_with_UserTHR")

In [5]:
data.shape

(2202660, 9)

In [6]:
data[:3]

,CellID,LocationX,LocationY,RSRP,RSRQ,SINR,Time,UserID,UserThR
0,1,361.511436,361.511436,-67.061741,NaN,86,0.2,1,86
1,2,361.511436,361.511436,-83.859050,NaN,77,0.2,1,77
2,3,361.511436,361.511436,-83.203628,NaN,15,0.2,1,15


In [7]:
testData = data[:1000]

In [8]:
wantedDF = pd.DataFrame()
i = 0
for stamp, group in testData.groupby(["Time", "UserID"]):
#     rsrp_top3 = [0,0,0]
#     rsrp_matched_record_index = [0, 0,0]
    rsrpIndexList = list()
    for i in range(group.shape[0]):
        rsrpIndexList.append((i, group["RSRP"][i]))
    rsrpIndexListSorted = sorted(rsrpIndexList, key=lambda x: x[1])
    row_rsrp1st = group[rsrpIndexListSorted[0][0]]
    wantedDF.iloc[i]["Time"] = row_rsrp1st["Time"]
    wantedDF.iloc[i]["User"] = row_rsrp1st["UserID"]
    wantedDF.iloc[i]["LocationX"] = row_rsrp1st["LocationX"]
    wantedDF.iloc[i]["LocationY"] = row_rsrp1st["LocationY"]
    wantedDF.iloc[i]["RSRP_1st"] = row_rsrp1st["RSRP"]
    wantedDF.iloc[i]["1stRSRP_Corresponding_RSRQ"] = row_rsrp1st["RSRQ"]
    wantedDF.iloc[i]["Serving_Cell"] = row_rsrp1st["CellID"]
    wantedDF.iloc[i]["RSRP_2nd"] = group[rsrpIndexListSorted[1][0]]["RSRP"]
    wantedDF.iloc[i]["2ndRSRP_Corresponding_RSRQ"] = group[rsrpIndexListSorted[1][0]]["RSRQ"]
    wantedDF.iloc[i]["RSRP_3rd"] = group[rsrpIndexListSorted[2][0]]["RSRP"]
    wantedDF.iloc[i]["3rdRSRP_Corresponding_RSRQ"] = group[rsrpIndexListSorted[2][0]]["RSRQ"]
    wantedDF.iloc[i]["RSRP_4th"] = group[rsrpIndexListSorted[3][0]]["RSRP"]
    wantedDF.iloc[i]["4thRSRP_Corresponding_RSRQ"] = group[rsrpIndexListSorted[3][0]]["RSRQ"]
    i += 1
    

KeyError: 4

In [2]:
def _connect_mongo():
    """ A util for making a connection to mongo """
    host = "localhost"
    port = 27017
    username = ""
    password = ""
    db = "5gopt"

    if username and password:
        mongo_uri = 'mongodb://%s:%s@%s:%s/%s' % (username, password, host, port, db)
        conn = MongoClient(mongo_uri)
    else:
        conn = MongoClient(host, port)


    return conn[db]

In [3]:
def collection_read_mongo(collection, query={}, no_id = True):
    db = _connect_mongo()
    cursor = db[collection].find(query)
    df = pd.DataFrame(list(cursor))

    if no_id:
        try:
            del df["_id"]
        except:
            pass
    return df

In [4]:
data = collection_read_mongo(collection="main_file_with_UserTHR")

In [5]:
data.shape

(2202660, 9)

In [6]:
data[:3]

,CellID,LocationX,LocationY,RSRP,RSRQ,SINR,Time,UserID,UserThR
0,1,361.511436,361.511436,-67.061741,NaN,86,0.2,1,86
1,2,361.511436,361.511436,-83.859050,NaN,77,0.2,1,77
2,3,361.511436,361.511436,-83.203628,NaN,15,0.2,1,15


In [7]:
testData = data[:1000]

In [8]:
groupedData = 

In [24]:
max(data["Throughput"])

0.0

In [25]:
groupedData = data.groupby(["Time", "CellID"])[["Throughput"]].sum()

In [26]:
type(groupedData)

pandas.core.frame.DataFrame

In [27]:
groupedData[:5]

Throughput
Time CellID            
0.2  0              0.0
0.4  1              0.0
     2              0.0
     4              0.0
     5              0.0

In [17]:
df = groupedData.to_frame

In [23]:
groupedData[36.2, 2]

0.0

In [28]:
groupedData["Throughput"]

Time  CellID
0.2   0         0.0
0.4   1         0.0
      2         0.0
      4         0.0
      5         0.0
      6         0.0
      7         0.0
      8         0.0
      9         0.0
0.6   1         0.0
      2         0.0
      4         0.0
      5         0.0
      6         0.0
      7         0.0
      8         0.0
      9         0.0
0.8   1         0.0
      2         0.0
      4         0.0
      5         0.0
      6         0.0
      7         0.0
      8         0.0
      9         0.0
1.0   1         0.0
      2         0.0
      4         0.0
      5         0.0
      6         0.0
               ... 
37.2  3         0.0
      4         0.0
      5         0.0
      6         0.0
      8         0.0
      9         0.0
37.4  1         0.0
      2         0.0
      3         0.0
      4         0.0
      5         0.0
      6         0.0
      8         0.0
      9         0.0
37.6  1         0.0
      2         0.0
      3         0.0
      4         0.0
      5